In [2]:
pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 75.9 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [19]:
df = pd.read_csv("corn yield.csv")
df

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2015,YEAR,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"245,000",NaN
1,SURVEY,2015,YEAR,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"36,015,000",NaN
2,SURVEY,2015,YEAR,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, SILAGE - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"9,000",NaN
3,SURVEY,2015,YEAR,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, SILAGE - YIELD, MEASURED IN TONS / ACRE",TOTAL,NOT SPECIFIED,15,NaN
4,SURVEY,2015,YEAR,NaN,STATE,ARIZONA,4.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"34,000",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23469,SURVEY,1866,YEAR,NaN,STATE,VIRGINIA,51.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"16,200,000",NaN
23470,SURVEY,1866,YEAR,NaN,STATE,WEST VIRGINIA,54.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"300,000",NaN
23471,SURVEY,1866,YEAR,NaN,STATE,WEST VIRGINIA,54.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"7,800,000",NaN
23472,SURVEY,1866,YEAR,NaN,STATE,WISCONSIN,55.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"500,000",NaN


In [21]:
df = df.drop(["Program","Week Ending","Ag District","Ag District Code","County","County ANSI","Zip Code","Region","Watershed","CV (%)","Domain Category","Commodity","Geo Level","watershed_code","Domain"],axis = 1)
df = df.drop(["State ANSI"],axis = 1)
df['Value'] = df['Value'].str.replace(',','')
df['Value'] = df['Value'].astype(float)

KeyError: "['Program', 'Week Ending', 'Ag District', 'Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'Watershed', 'CV (%)', 'Domain Category', 'Commodity', 'Geo Level', 'watershed_code', 'Domain'] not found in axis"

In [22]:
def one_hot_encoding(df,column):
    one_hot_encoder=ce.OneHotEncoder(cols=column,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [23]:
df = one_hot_encoding(df,"Period")
df = one_hot_encoding(df,"Data Item")
df = one_hot_encoding(df,"State")

In [24]:
random_reg = RandomForestRegressor(n_estimators=100,random_state=27)

In [25]:
X = df.drop("Value",axis = 1)
Y = df["Value"]

In [26]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=27,test_size=0.3)

In [15]:
# train,test = train_test_split(df, test_size=0.5,random_state= 27)

In [27]:
random_reg.fit(X_train, Y_train)

RandomForestRegressor(random_state=27)

In [28]:
Y_pred = random_reg.predict(X_test)

In [ ]:

import joblib

joblib.dump(random_reg, 'corn_yield_model.joblib')